<a href="https://colab.research.google.com/github/SuchV/MEDICA/blob/main/Colab_Test_Medica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install flask flask-sqlalchemy

# Import libraries
from flask import Flask, request, render_template_string, redirect, url_for, session
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash, check_password_hash
import threading

# Set up Flask application (only define this ONCE)
app = Flask(__name__)
app.secret_key = 'your_secret_key'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///patients.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# Define Patient model
class Patient(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    email = db.Column(db.String(100), unique=True, nullable=False)
    password = db.Column(db.String(200), nullable=False)
    age = db.Column(db.Integer)
    gender = db.Column(db.String(10))
    blood_type = db.Column(db.String(5))
    medical_history = db.Column(db.Text)

# Create the database tables
with app.app_context():
    db.create_all()

    # Check if test user exists
    test_user = Patient.query.filter_by(email='test@example.com').first()
    if not test_user:
        # Create a test user for login
        test_password = generate_password_hash('password123')
        test_patient = Patient(
            name='Jan Kowalski',
            email='test@example.com',
            password=test_password,
            age=30,
            gender='Male',
            blood_type='O+',
            medical_history='To jest testowe konto z przykładową historią medyczną.'
        )
        db.session.add(test_patient)
        db.session.commit()
        print("Utworzono testowego użytkownika: email=test@example.com, hasło=password123")

# Formularz rejestracji pacjenta
registration_form = """
<!DOCTYPE html>
<html>
<head>
    <title>Medica+ | Rejestracja Pacjenta</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 20px; background-color: #f5f5f5; }
        .container { max-width: 600px; margin: 0 auto; background: white; padding: 20px; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
        h1 { text-align: center; color: #2c3e50; }
        .logo { text-align: center; margin-bottom: 20px; color: #3498db; font-size: 24px; font-weight: bold; }
        .form-group { margin-bottom: 15px; }
        label { display: block; margin-bottom: 5px; font-weight: bold; }
        input, select, textarea { width: 100%; padding: 8px; border: 1px solid #ddd; border-radius: 4px; box-sizing: border-box; }
        button { background-color: #3498db; color: white; border: none; padding: 10px 15px; border-radius: 4px; cursor: pointer; }
        button:hover { background-color: #2980b9; }
        .message { padding: 10px; margin-bottom: 15px; border-radius: 4px; }
        .success { background-color: #d4edda; color: #155724; }
        .error { background-color: #f8d7da; color: #721c24; }
        .login-link { text-align: center; margin-top: 20px; }
    </style>
</head>
<body>
    <div class="container">
        <div class="logo">Medica+</div>
        <h1>Rejestracja Pacjenta</h1>
        {% if message %}
        <div class="message {% if success %}success{% else %}error{% endif %}">
            {{ message }}
        </div>
        {% endif %}
        <form method="POST" action="/register">
            <div class="form-group">
                <label for="name">Imię i nazwisko:</label>
                <input type="text" id="name" name="name" required>
            </div>
            <div class="form-group">
                <label for="email">Email:</label>
                <input type="email" id="email" name="email" required>
            </div>
            <div class="form-group">
                <label for="password">Hasło:</label>
                <input type="password" id="password" name="password" required>
            </div>
            <div class="form-group">
                <label for="age">Wiek:</label>
                <input type="number" id="age" name="age" required>
            </div>
            <div class="form-group">
                <label for="gender">Płeć:</label>
                <select id="gender" name="gender" required>
                    <option value="Male">Mężczyzna</option>
                    <option value="Female">Kobieta</option>
                    <option value="Other">Inna</option>
                </select>
            </div>
            <div class="form-group">
                <label for="blood_type">Grupa krwi:</label>
                <select id="blood_type" name="blood_type">
                    <option value="A+">A+</option>
                    <option value="A-">A-</option>
                    <option value="B+">B+</option>
                    <option value="B-">B-</option>
                    <option value="AB+">AB+</option>
                    <option value="AB-">AB-</option>
                    <option value="O+">O+</option>
                    <option value="O-">O-</option>
                </select>
            </div>
            <div class="form-group">
                <label for="medical_history">Historia medyczna:</label>
                <textarea id="medical_history" name="medical_history" rows="4"></textarea>
            </div>
            <button type="submit">Zarejestruj się</button>
        </form>
        <div class="login-link">
            Masz już konto? <a href="/login">Zaloguj się tutaj</a>
        </div>
    </div>
</body>
</html>
"""

# Formularz logowania pacjenta
login_form = """
<!DOCTYPE html>
<html>
<head>
    <title>Medica+ | Logowanie Pacjenta</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 20px; background-color: #f5f5f5; }
        .container { max-width: 400px; margin: 0 auto; background: white; padding: 20px; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
        h1 { text-align: center; color: #2c3e50; }
        .logo { text-align: center; margin-bottom: 20px; color: #3498db; font-size: 24px; font-weight: bold; }
        .form-group { margin-bottom: 15px; }
        label { display: block; margin-bottom: 5px; font-weight: bold; }
        input { width: 100%; padding: 8px; border: 1px solid #ddd; border-radius: 4px; box-sizing: border-box; }
        button { background-color: #3498db; color: white; border: none; padding: 10px 15px; border-radius: 4px; cursor: pointer; width: 100%; }
        button:hover { background-color: #2980b9; }
        .message { padding: 10px; margin-bottom: 15px; border-radius: 4px; }
        .error { background-color: #f8d7da; color: #721c24; }
        .register-link { text-align: center; margin-top: 20px; }
    </style>
</head>
<body>
    <div class="container">
        <div class="logo">Medica+</div>
        <h1>Logowanie Pacjenta</h1>
        {% if error %}
        <div class="message error">
            {{ error }}
        </div>
        {% endif %}
        <form method="POST" action="/login">
            <div class="form-group">
                <label for="email">Email:</label>
                <input type="email" id="email" name="email" required>
            </div>
            <div class="form-group">
                <label for="password">Hasło:</label>
                <input type="password" id="password" name="password" required>
            </div>
            <button type="submit">Zaloguj się</button>
        </form>
        <div class="register-link">
            Nie masz konta? <a href="/register">Zarejestruj się tutaj</a>
        </div>
    </div>
</body>
</html>
"""

# Karta pacjenta
patient_card = """
<!DOCTYPE html>
<html>
<head>
    <title>Medica+ | Karta Pacjenta</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 0; padding: 20px; background-color: #f5f5f5; }
        .container { max-width: 800px; margin: 0 auto; background: white; padding: 20px; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
        .logo { text-align: center; margin-bottom: 20px; color: #3498db; font-size: 24px; font-weight: bold; }
        h1 { text-align: center; color: #2c3e50; margin-bottom: 30px; }
        .patient-info { margin-bottom: 30px; }
        .patient-info h2 { color: #3498db; border-bottom: 1px solid #eee; padding-bottom: 10px; }
        .info-row { display: flex; margin-bottom: 10px; }
        .info-label { flex: 0 0 150px; font-weight: bold; }
        .info-value { flex: 1; }
        .medical-history { margin-top: 20px; }
        .medical-history h3 { color: #2c3e50; }
        .medical-history p { line-height: 1.6; }
        .logout { display: block; text-align: center; margin-top: 30px; }
        .logout a { display: inline-block; background-color: #e74c3c; color: white; padding: 10px 15px; text-decoration: none; border-radius: 4px; }
        .logout a:hover { background-color: #c0392b; }
    </style>
</head>
<body>
    <div class="container">
        <div class="logo">Medica+</div>
        <h1>Karta Medyczna Pacjenta</h1>

        <div class="patient-info">
            <h2>Dane Osobowe</h2>
            <div class="info-row">
                <div class="info-label">Imię i nazwisko:</div>
                <div class="info-value">{{ patient.name }}</div>
            </div>
            <div class="info-row">
                <div class="info-label">Email:</div>
                <div class="info-value">{{ patient.email }}</div>
            </div>
            <div class="info-row">
                <div class="info-label">Wiek:</div>
                <div class="info-value">{{ patient.age }}</div>
            </div>
            <div class="info-row">
                <div class="info-label">Płeć:</div>
                <div class="info-value">
                    {% if patient.gender == "Male" %}Mężczyzna
                    {% elif patient.gender == "Female" %}Kobieta
                    {% else %}Inna{% endif %}
                </div>
            </div>
            <div class="info-row">
                <div class="info-label">Grupa krwi:</div>
                <div class="info-value">{{ patient.blood_type }}</div>
            </div>
        </div>

        <div class="medical-history">
            <h3>Historia leczenia</h3>
            <p>{{ patient.medical_history or 'Brak dostępnej historii medycznej.' }}</p>
        </div>

        <div class="logout">
            <a href="/logout">Wyloguj się</a>
        </div>
    </div>
</body>
</html>
"""

# Function to find an available port
import socket
def find_available_port(start_port=5000, max_port=9000):
    for port in range(start_port, max_port):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('localhost', port)) != 0:
                return port
    return 8080  # Fallback port

# Simplified application runner without reloader issues
def start_flask_app():
    port = find_available_port()
    print(f"Starting Flask app on port {port}")

    # Disable reloader and run in current thread
    app.run(host='0.0.0.0', port=port, debug=True, use_reloader=False)
    return port

# Create a special endpoint for the home page if not defined
@app.route('/')
def home():
    if 'user_id' in session:
        return redirect(url_for('patient_page'))
    else:
        return redirect(url_for('login'))

# Create a patient page route if not already defined
@app.route('/patient')
def patient_page():
    if 'user_id' not in session:
        return redirect(url_for('login'))

    patient = Patient.query.get(session['user_id'])
    if patient:
        return render_template_string(patient_card, patient=patient)
    else:
        return redirect(url_for('logout'))

# Create logout route if not already defined
@app.route('/logout')
def logout():
    session.pop('user_id', None)
    return redirect(url_for('login'))

@app.route('/login', methods=['GET', 'POST'])
def login():
    error = None

    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']

        patient = Patient.query.filter_by(email=email).first()

        if patient and check_password_hash(patient.password, password):
            session['user_id'] = patient.id
            return redirect(url_for('patient_page'))
        else:
            error = "Nieprawidłowy email lub hasło. Spróbuj ponownie."

    return render_template_string(login_form, error=error)

@app.route('/register', methods=['GET', 'POST'])
def register():
    message = None
    success = False

    if request.method == 'POST':
        name = request.form['name']
        email = request.form['email']
        password = request.form['password']
        age = int(request.form['age'])
        gender = request.form['gender']
        blood_type = request.form['blood_type']
        medical_history = request.form['medical_history']

        # Check if email already exists
        existing_patient = Patient.query.filter_by(email=email).first()
        if existing_patient:
            message = "Email jest już zarejestrowany. Proszę użyć innego adresu email."
        else:
            hashed_password = generate_password_hash(password)
            new_patient = Patient(name=name, email=email, password=hashed_password,
                                age=age, gender=gender, blood_type=blood_type,
                                medical_history=medical_history)
            db.session.add(new_patient)
            db.session.commit()
            message = "Rejestracja zakończona sukcesem! Możesz się teraz zalogować."
            success = True

    return render_template_string(registration_form, message=message, success=success)


# Use a simpler approach without threading complications
port = find_available_port()
print(f"Starting Flask application on port {port}")

# Start the app in a thread but without reloader
import threading
threading.Thread(target=lambda: app.run(
    host='0.0.0.0',
    port=port,
    debug=True,
    use_reloader=False  # This is the key fix
)).start()

# Wait for the app to start
import time
time.sleep(2)

# Display the app in an iframe
from google.colab import output
output.serve_kernel_port_as_iframe(port, width=1000, height=800)

print("Application is running!")
print("Test login credentials: email=test@example.com, password=password123")

Starting Flask application on port 5004
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5004
 * Running on http://172.28.0.12:5004
INFO:werkzeug:Press CTRL+C to quit


<IPython.core.display.Javascript object>

Application is running!
Test login credentials: email=test@example.com, password=password123
